In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
torch.get_num_threads()

6

## 1 Prepare Dataset

In [22]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [23]:
# load text dataset
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

print("length of text: {} characters".format(len(text)))
print(text[:200])

length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [24]:
# list all the chars in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("total chars: ", vocab_size)
print(" ".join(chars))

total chars:  65

   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z


In [25]:
# tokenize the text
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
token_encode = lambda s: [char_to_ix[c] for c in s]
token_decode = lambda i: "".join([ix_to_char[c] for c in i])

print("encoded text: ", token_encode("hii there!"))
print("decode text: ", token_decode(token_encode("hii there!")))

encoded text:  [46, 47, 47, 1, 58, 46, 43, 56, 43, 2]
decode text:  hii there!


In [26]:
# convert text to tensor
data = torch.tensor(token_encode(text), dtype=torch.long).to(device)
print(data.shape, data.dtype)
print(data[:100])

# split data into train and test
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [27]:
# example of chunk of dataset
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")

when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [35]:
# example of how to create a batch of data
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(f"inputs: {xb.shape}")
print(xb)
print(f"targets: {yb.shape}")
print(yb)
print("-"*30)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context}, the target is {target}")
    print("="*30)

inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------
when input is tensor([24]), the target is 43
when input is tensor([24, 43]), the target is 58
when input is tensor([24, 43, 58]), the target is 5
when input is tensor([24, 43, 58,  5]), the target is 57
when input is tensor([24, 43, 58,  5, 57]), the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when input is tensor([44]), the target is 53
when input is tensor([44, 53]), the target is 56
when input i

## 2 Simplest Model: Bigram Model

In [51]:
# bigram model
torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_tabel = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, inputs, targets=None):
        # inputs: (batch_size, block_size)
        # targets: (batch_size, block_size)
        logits = self.token_embedding_tabel(inputs) # (batch_size, block_size, vocab_size) / (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits_rs = logits.view(B*T, C)
            targets_rs = targets.view(B*T)
            # loss = F.cross_entropy(logits.transpose(1,2), targets)
            loss = F.cross_entropy(logits_rs, targets_rs)
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx: (batch_size, block_size) array of indices in the current context
        for _ in range(max_new_token):
            logits, loss = self(idx) # (batch_size, block_size, vocab_size)
            # only focus on the last token
            last_logits = logits[:, -1, :]
            probs = F.softmax(last_logits, dim=-1) # (batch_size, vocab_size)
            idx_next = torch.multinomial(input=probs, num_samples=1) # sampling: (batch_size, 1)
            idx = torch.cat([idx, idx_next], dim=1) # (batch_size, block_size+1)
        
        return idx
                    
# test bigram model
model1 = BigramModel(vocab_size).to(device)
logits, loss = model1(xb, yb)
print(logits.shape, loss)


idx_init = torch.zeros((1,1), dtype=torch.long).to(device)
print(token_decode(model1.generate(idx_init, max_new_token=100)[0].tolist()))


torch.Size([4, 8, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [57]:
model1 = BigramModel(vocab_size).to(device)

# train above model
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
batch_size = 32
for iter in range(10000):
    xb, yb = get_batch("train")
    logits, loss = model1(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f"epoch: {iter}, loss: {loss.item():.3f}")
print(token_decode(model1.generate(idx_init, max_new_token=300)[0].tolist()))

epoch: 9999, loss: 2.446

hayoince al'-wie ry mUThin meeancourenotyeisomy t VI:
COMEEPEPEren p:
Kad, thicYCHMI pashad a here, RDY ararroutyer'd,
Bearnd me lef;
MELI d,
QUMe gongry RAlise monguee ces g nchee Iss tast theran, s!
FJaboaingednvfonghe; bold o ouleareruk ceee.
The prnt Whandea at, avenan
OUSino th
Mainol.
Five OHa


## 3 Self-Attention 
Different from the attention used in encoder-decoder structure, the key, query, and value here all come from the same source, i.e., the input sequence. That's why it calls self-attention. The cross attention will have separate source of nodes to pull information from.

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [98]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = torch.matmul(q, k.transpose(-2,-1)) # (B, T, T), each row of wei[i] is a query vector (weights of each token)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, head_size)
out = torch.matmul(wei, v) # (B, T, head_size)
print(wei[0])
print(out.shape)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)
torch.Size([4, 8, 16])


In [96]:
tril

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.]])

In [32]:
input = torch.randn(3, 5, requires_grad=True)
input

tensor([[ 1.3863, -2.0786,  1.1903, -0.4894, -1.4285],
        [ 0.0935,  1.1171,  1.5847, -0.0534,  0.6406],
        [ 0.8000, -1.9087,  0.7417,  1.1109,  0.0096]], requires_grad=True)

In [33]:
target = torch.randint(5, (3,), dtype=torch.int64)
target

tensor([3, 4, 4])

In [34]:
loss = F.cross_entropy(input, target)
loss

tensor(2.1895, grad_fn=<NllLossBackward0>)

## Appendix

#### Attention
##### Why is it important to have scale step in attention?
Without scale (i.e., using sqrt(head_size)), the variance of attention weights will be equal to the head_size. For softmax function, if the weight takes very positive or very negative numbers, the softmax output will actually converge to one-hot vector. But we need the coefficents to be diffuse.

In [109]:
# example of softmax
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.2, 0.5]), dim=-1))
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.2, 0.5])*30, dim=0))

tensor([0.1799, 0.1333, 0.2197, 0.1988, 0.2684])
tensor([6.1282e-06, 7.5628e-10, 2.4723e-03, 1.2309e-04, 9.9740e-01])


In [105]:
torch.manual_seed(1337)
B, T, C = 1, 8, 32
x = torch.randn(B, T, C)

# single head perform self-attention
head_size = 16

# transform from int to float
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = torch.matmul(q, k.transpose(-2,-1)) # (B, T, T), each row of wei[i] is a query vector (weights of each token)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
print("without normalization:")
print(wei[0])
print("softmax:")
print(F.softmax(wei, dim=-1)[0])

print("=====================================")
print("with normalization:")
print(wei[0]/torch.sqrt(torch.tensor(head_size).long()))
print("softmax:")
print(F.softmax(wei/torch.sqrt(torch.tensor(head_size).long()), dim=-1)[0])

without normalization:
tensor([[-2.6915,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3898,  0.3375,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.9337,  1.7324,  0.3765,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.9253,  0.2274,  0.7229,  0.5320,    -inf,    -inf,    -inf,    -inf],
        [-0.7654, -2.0773,  0.2287,  1.8543, -1.6185,    -inf,    -inf,    -inf],
        [-3.8860,  0.2991, -1.5944, -0.2074, -3.0268, -0.3258,    -inf,    -inf],
        [-0.0754,  2.7371,  0.9082,  1.8673, -0.0628,  1.1669,  0.6911,    -inf],
        [-1.6482, -0.1062,  1.5168,  2.8792,  0.2492,  0.4291,  0.6414, -1.0879]],
       grad_fn=<SelectBackward0>)
softmax:
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0235, 0.9765, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0524, 0.7534, 0.1942, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3345, 0.1665, 0.2732, 0.2258, 0.0000, 0.0000

#### The Matematical Trick in Self-Attention

In [86]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# version 1: bag of words using for loop
xbow_loop = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1,:] # (t, C)
        xbow_loop[b,t] = torch.mean(x_prev, dim=0)

xbow_mat1 = torch.zeros(B, T, C)

# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
for b in range(B):
    xbow_mat1[b] = torch.matmul(wei, x[b])
print(torch.allclose(xbow_loop, xbow_mat1, atol=1e-7))

# version 3
xbow_mat2 = torch.matmul(wei, x)
# xbow_mat2 = wei @ x # (T, T) @ (B, T, C) = (B, T, C)
print(torch.allclose(xbow_loop, xbow_mat2, atol=1e-7))
print(wei)

True
True
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


$$sofxmax(\mathbf{z})_i=\frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$$

In [89]:
# version 4: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril==0, float("-inf"))
print(wei)
wei = F.softmax(wei, dim=-1)
print(wei)
xbow_mat3 = torch.matmul(wei, x)
print(torch.allclose(xbow_loop, xbow_mat3, atol=1e-7))

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
True


In [70]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True) # average
b = torch.randint(0,10,(3,2)).float()
c1 = a @ b
c2 = torch.matmul(a, b)
print("a=")
print(a)
print("b=")
print(b)
print("c1=")
print(c1)
print("c2=")
print(c2)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c1=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
c2=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [94]:
block_size = 10
embedding_dim = 5
a = torch.arange(block_size)
print(a.shape)
emb = nn.Embedding(block_size, embedding_dim)
emb(a).shape

torch.Size([10])


torch.Size([10, 5])

In [ ]:
# torch,gather function
input = torch.randint(0, 10, size=(3,4)).long()
index = torch.randint(0, 3, size=(2,4)).long()
gather_dim0 = torch.gather(input, dim=0, index=index)
gather_dim1 = torch.gather(input, dim=1, index=index)
print(f"input:\n {input}")
print(f"index\n: {index}")
print(f"gather (dim=0):\n {gather_dim0}")
print(f"gather (dim=1):\n {gather_dim1}")

In [ ]:
# cross entropy loss
def compute_cross_entropy(input, target):
    # input: (batch_size, block_size, vocab_size): vocab_size is the number of classes, i.e., it contains the probability of each class
    # target: (batch_size, block_size)
    # return: (batch_size, block_size)
    input = torch.softmax(input, dim=2)
    p = torch.gather(input, dim=2, index=target.unsqueeze(2)).squeeze(2) # batch_size, block_size
    loss = torch.mean(-torch.log(p))
    return loss

torch.manual_seed(0)
word_to_idx = {"cat": 0, "dog": 1, "bird": 2, "fish": 3}
vs = len(word_to_idx)
bs = 2 # batch size
ts = 3 # block size
embedding_dim = 4
embedding = nn.Embedding(vs, embedding_dim)
print(f"embedding wight:\n {embedding.weight}")

input_index = torch.randint(0, vs, size=(bs,ts)).long()
target_index = torch.randint(0, vs, size=(bs,ts)).long()

print(f"input index:\n {input_index}")
print(f"target index:\n {target_index}")

embed_out = embedding(input_index)
print(f"embed out:\n {embed_out.shape}") # batch_size, block_size, embedding_dim
print(embed_out)

loss1 = F.cross_entropy(embed_out.transpose(1,2), target_index)
loss2 = compute_cross_entropy(embed_out, target_index)
print(f"loss1: {loss1}, loss2: {loss2}")



In [45]:
# torch.multinomial
weights = torch.tensor([0.1, 0.2, 0.3, 0.4])
sampled_index = torch.multinomial(weights, 1)

# The result is a tensor containing the index of the randomly chosen item
# For example, if the result is 2, it means that the item "orange" was chosen
print(sampled_index.item())


1


In [4]:
# batch normalization
class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1, training=True):
        self.eps = eps
        self.momentum = momentum
        self.training = training
        self.gamma = torch.ones(dim) # layer norm gain
        self.beta = torch.zeros(dim) # layer norm bias
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    
    def __call__(self, x):
        # x: (batch_size, block_size, embed_size)
        if self.training:
            xmean = x.mean(dim=-1, keepdim=True) # batch mean
            xvar = x.var(dim=-1, keepdim=True) # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize
        self.out = self.gamma * xhat + self.beta # scale and shift
        
        # update running mean and variance
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]



In [18]:
dim = 100
bs = 32
module = BatchNorm1d(dim=dim)
x = torch.randn(bs, dim)
x = module(x)
print(x.shape)
print(x[:,0].mean())
print(x[:,0].var())
print("-------")
print(x[0,:].mean())
print(x[0,:].var())

torch.Size([32, 100])
tensor(0.2217)
tensor(1.4339)
-------
tensor(8.9407e-09)
tensor(1.0000)


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.distributed as dist
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp


# load text dataset
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
# list all the chars in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from char to index
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
token_encoder = lambda s: [stoi[c] for c in s]
token_decoder = lambda l: ''.join([itos[i] for i in l])

# creat dataset for train and val data
class TextDataset(Dataset):
    def __init__(self, data, block_size):
        # self.data = torch.tensor(data)
        self.data = data
        self.block_size = block_size
        self.num_blocks = len(self.data) // self.block_size
    
    def __len__(self):
        return self.num_blocks
    
    def __getitem__(self, index):
        x = self.data[index * self.block_size: (index + 1) * self.block_size]
        y = self.data[index * self.block_size + 1: (index + 1) * self.block_size + 1]
        return x, y
    


model_paras = {'embedding_size': 384,
               'head_size': 64,
               'vocab_size': vocab_size,
               'block_size': 8,
               'num_heads': 6,
               'num_layers': 6,
               'dropout': 0.2,
               'lr': 3e-4,
               'batch_size': 32,
               'num_epochs': 10}


data = token_encoder(text)
# convert to tensor
data = torch.tensor(data, dtype=torch.long)
train_data = data[:int(len(data)*0.9)]
train_set = TextDataset(train_data, model_paras['block_size'])
val_data = data[int(len(data)*0.9):]
val_set = TextDataset(val_data, model_paras['block_size'])

train_loader = DataLoader(train_set, batch_size=model_paras['batch_size'], shuffle=True)
val_loader = DataLoader(val_set, batch_size=model_paras['batch_size'], shuffle=False)
num_batches = len(train_loader)
print(f"num_batches in train: {num_batches}")
print(f"num_batches in val: {len(val_loader)}")


num_batches in train: 3922
num_batches in val: 436


In [14]:
mp.cpu_count()

12

To modify the main_worker function to include validation after each epoch and return the final trained model, you'll need to make a few changes. First, you'll need to pass a validation dataset and DataLoader to the main_worker function. You'll also need to use a Queue from the multiprocessing module to return the trained model from each process. Here's a modified version of the main_worker function:

```
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.utils.data
from torch.utils.data import DataLoader, DistributedSampler

def main_worker(rank, world_size, model, train_dataset, val_dataset, batch_size, epochs, queue):
    # Setup
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

    # Create train sampler and DataLoader
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)

    # Create validation DataLoader
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # Move the model to the current device and wrap with DDP
    device = torch.device(f"cpu:{rank}")
    model.to(device)
    ddp_model = DDP(model, device_ids=[device], output_device=device)

    # Training and validation loop
    for epoch in range(epochs):
        # Training
        ddp_model.train()
        train_sampler.set_epoch(epoch)
        for batch_idx, batch in enumerate(train_dataloader):
            # ... (training code)

        # Validation
        ddp_model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                # ... (validation code)
        
    # Save the trained model in the queue
    queue.put(model.state_dict())

# Initialize the model, datasets, and multiprocessing queue
model = ... # Your model definition
train_dataset = ... # Your train dataset definition
val_dataset = ... # Your validation dataset definition
queue = mp.Queue()

# Create and start the processes
processes = []
world_size = 8
for rank in range(world_size):
    p = mp.Process(target=main_worker, args=(rank, world_size, model, train_dataset, val_dataset, batch_size, epochs, queue))
    p.start()
    processes.append(p)

# Collect the trained models from each process
trained_models = []
for _ in range(world_size):
    trained_models.append(queue.get())

# Join the processes
for p in processes:
    p.join()

# Do something with the trained models, e.g., average their parameters or select the best one based on validation performance

```


In this modified main_worker function, we pass the validation dataset, create a validation DataLoader, and add a validation loop after the training loop in each epoch. To return the final trained model, we use a multiprocessing Queue to collect the models' state dictionaries from each process.

Once you have the trained models, you can decide what to do with them, such as averaging their parameters or selecting the best model based on validation performance.